# Geekbench benchmark on Android

Geekbench4 is an app offering several benchmarks to run on android smartphones. The one used in this notebook is the '**CPU**' benchmark, which runs several workloads that follow the lines of what is commonly run by smartphones (AES, JPEG codec, FFT, and so on). The benchmark runs all the tests in '**Single-Core**' mode as well as in '**Multi-Core**' in order to compare the single-thread and multi-thread performances of the device.

**Do note that the benchmark will attempt to upload its results, which includes some hardware information**

In [1]:
from conf import LisaLogging
LisaLogging.setup()

2017-08-08 10:44:57,487 INFO    : root         : Using LISA logging configuration:
2017-08-08 10:44:57,489 INFO    : root         :   /media/build/workspace/lisa-github/logging.conf


In [2]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Support Functions

This function helps us run our experiments:

In [3]:
def experiment():
    
    # Configure governor
    target.cpufreq.set_all_governors('sched')
    
    # Get workload
    wload = Workload.getInstance(te, 'Geekbench')
    
    # Run Geekbench workload
    wload.run(te.res_dir, test_name='CPU', collect='ftrace')
        
    # Dump platform descriptor
    te.platform_dump(te.res_dir)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [4]:
# Setup target configuration
my_conf = {

#     # Target platform and board
#     "platform"     : 'android',
#     "board"        : 'pixel',
    
#     # Device
#     "device"       : "0123456789ABCDEF",
    
#     # Android home
#     "ANDROID_HOME" : "/home/vagrant/lisa/tools/android-sdk-linux/",
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/media/build/workspace/android-sdk",

    # Folder where all the results will be collected
    "results_dir" : datetime.datetime.now()\
                    .strftime("Geekbench_example_" + '%Y%m%d_%H%M%S'),

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_overutilized",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "cpu_capacity",
            "cpu_frequency",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset'],
}

In [5]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False)
target = te.target

2017-08-08 10:45:16,190 INFO    : TestEnv      : Using base path: /media/build/workspace/lisa-github
2017-08-08 10:45:16,198 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-08-08 10:45:16,203 INFO    : TestEnv      : External tools using:
2017-08-08 10:45:16,204 INFO    : TestEnv      :    ANDROID_HOME: /media/build/workspace/android-sdk
2017-08-08 10:45:16,204 INFO    : TestEnv      :    CATAPULT_HOME: /media/build/workspace/lisa-github/tools/catapult
2017-08-08 10:45:16,208 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
2017-08-08 10:45:16,210 INFO    : TestEnv      : Connecting Android target [0123456789ABCDEF]
2017-08-08 10:45:16,212 INFO    : TestEnv      : Connection settings:
2017-08-08 10:45:16,214 INFO    : TestEnv      :    {'device': '0123456789ABCDEF'}
2017-08-08 10:45:17,114 INFO    : TestEnv      : Initializing target workdir:
2017-08-08 10:45:17,121 INFO    : TestEnv      :    /data/local/tmp/devlib-target
2017-08

## Workloads execution

This is done using the **experiment** helper function defined above which is configured to run a **Geekbench - CPU** experiment.

In [6]:
# Initialize Workloads for this test environment
results = experiment()

2017-08-08 10:45:38,183 INFO    : Workload     : Supported workloads available on target:
2017-08-08 10:45:38,190 INFO    : Workload     :   viewerworkload, uibench, jankbench, geekbench
2017-08-08 10:45:39,773 INFO    : Screen       : Force manual orientation
2017-08-08 10:45:39,774 INFO    : Screen       : Set orientation: PORTRAIT
2017-08-08 10:45:41,184 INFO    : Screen       : Set brightness: 0%
2017-08-08 10:45:41,198 INFO    : System       : am start -n com.primatelabs.geekbench .HomeActivity
2017-08-08 10:45:43,543 INFO    : Geekbench    : adb -s 0123456789ABCDEF logcat ActivityManager:* System.out:I *:S GEEKBENCH_RESULT:*
2017-08-08 10:45:43,564 INFO    : Geekbench    : send tap event
2017-08-08 10:45:43,567 INFO    : System       : input tap 1179 555
2017-08-08 10:45:44,937 INFO    : System       : input tap 1472 320
2017-08-08 10:45:45,319 INFO    : Geekbench    : FTrace START


2017-08-08 10:51:01,976 INFO    : Geekbench    : FTrace STOP
2017-08-08 10:51:28,832 INFO    : Screen       : Set orientation: AUTO
2017-08-08 10:51:29,918 INFO    : Screen       : Set brightness: AUTO


## Results analysis

Geekbench4 uses a baseline score of 4000, which is the benchmark score of an Intel Core i7-6600U. Higher scores are better, with double the score indicating double the performance. You can have a look at the results for several android phones here https://browser.primatelabs.com/android-benchmarks

In [7]:
class Geekbench(object):
    """
    Geekbench json results parsing class
    """
    def __init__(self, filepath):
        with open(filepath) as fd:
            self.__json = json.loads(fd.read())
        
        self.benchmarks = {}
        for section in self.__json["sections"]:
            self.benchmarks[section["name"]] = section
            for workload in section["workloads"]:
                self.benchmarks[section["name"]][workload["name"]] = workload     
            
    def name(self):
        """Get a human-readable name for the geekbench run
        """
        gov = ""
        build = ""
        for metric in self.__json["metrics"]:
            if metric["name"] == "Governor":
                gov = metric["value"]
            elif metric["name"] == "Build":
                build = metric["value"]

        return "[build]=\"{}\" [governor]=\"{}\"".format(build, gov)
    
    def benchmarks_names(self):
        """Get a list of benchmarks (e.g. Single-Core, Multi-Core) found in the run results        
        """
        return [section["name"] for section in self.__json["sections"]]
    
    def workloads_names(self):
        """Get a list of unique workloads (e.g. EAS, Dijkstra) found in the run results
        """
        return [workload["name"] for workload in self.benchmarks.values()[0]["workloads"]]
    
    def global_scores(self):
        """Get the overall scores of each benchmark
        """
        data = {}
        for benchmark in self.benchmarks_names():
            data[benchmark] = self.benchmarks[benchmark]["score"]
        return data
        
    def detailed_scores(self):
        """Get the detailed workload scores of each benchmark
        """
        benchmark_fields = ["score", "runtime_mean", "rate_string"]
        benches = {}
        benchmarks = self.benchmarks_names()
        workloads = self.workloads_names() 
        
        for benchmark in benchmarks:
            data = {}
            for workload in workloads:
                data[workload] = {}
                for field in benchmark_fields:
                    data[workload][field] = self.benchmarks[benchmark][workload][field]        
            benches[benchmark] = data
            
        return benches

In [8]:
def display_bench_results(geekbench, detailed=False):
    print "===== Global results ====="
    
    scores = geekbench.global_scores()
    
    # Build dataframe for display
    row = []
    for bench_type, score in scores.iteritems():
        row.append(score)
        
    df = pd.DataFrame(data=row, index=scores.keys(), columns=["Global score"])
    display(df)
    
    if not detailed:
        return
    
    print "===== Detailed results ====="
    
    scores = geekbench.detailed_scores()
    
    for benchmark, results in geekbench.detailed_scores().iteritems():
        print "----- {} benchmark -----".format(benchmark)
        # Build dataframe for display
        data = []
        idx = []
        columns = results.values()[0].keys()
        for workload, fields in results.iteritems():
            data.append(tuple(fields.values()))
            idx.append(workload)
        display (pd.DataFrame(data=data, index=idx, columns=columns))

In [9]:
for f in os.listdir(te.res_dir):
    if f.endswith(".gb4"):
        geekbench = Geekbench(te.res_dir + "/" + f)
        
        print "Analysing geekbench {}".format(geekbench.name())
        display_bench_results(geekbench, True)

Analysing geekbench [build]="hikey960-userdebug O NYC eng.stemia.20170804.094239 test-keys" [governor]="sched"
===== Global results =====


,Global score
Single-Core,1905
Multi-Core,3931


===== Detailed results =====
----- Single-Core benchmark -----


,score,runtime_mean,rate_string
AES,1176,0.142312,907.2 MB/sec
HDR,2102,0.568124,7.62 Mpixels/sec
Rigid Body Physics,2036,0.252157,5962.4 FPS
HTML5 Parse,1970,0.137031,8.95 MB/sec
Lua,1692,0.184388,1.74 MB/sec
Camera,2025,0.180315,5.62 images/sec
Histogram Equalization,1654,0.205208,51.7 Mpixels/sec
SQLite,1899,0.384648,52.7 Krows/sec
Face Detection,1741,0.149000,508.7 Ksubwindows/sec
Memory Copy,1509,0.478858,4.18 GB/sec


----- Multi-Core benchmark -----


,score,runtime_mean,rate_string
AES,3391,0.433510,2.55 GB/sec
HDR,5549,1.795542,20.1 Mpixels/sec
Rigid Body Physics,3911,1.156668,11450.2 FPS
HTML5 Parse,5521,0.439192,25.1 MB/sec
Lua,4625,0.578663,4.75 MB/sec
Camera,5369,0.570530,14.9 images/sec
Histogram Equalization,4569,0.585957,142.8 Mpixels/sec
SQLite,4604,1.289821,127.6 Krows/sec
Face Detection,5022,0.481703,1.47 Msubwindows/sec
Memory Copy,2221,0.679739,6.15 GB/sec


## Analysing several runs

It can be interesting to compare Geekbench results with different parameters (kernel, drivers) and even different devices to gauge the impact of these parameters. As Geekbench results can vary a bit from one run to another, having a set of repeated runs is preferable.

The following section will grab the results of all the **Geekbench\_exemple\_\*** results found in the LISA results directory

In [10]:
import glob

def fetch_results():
    results_path = os.path.join(te.LISA_HOME, "results")
    
    results_dirs = [results_path + "/" + d for d in os.listdir(results_path) if d.startswith("Geekbench_example_")]
    
    res = []
    
    for d in results_dirs:
        bench_file = glob.glob("{}/*.gb4".format(d))[0]
        res.append(Geekbench(bench_file))
        
    return res

def compare_runs():
    geekbenches = fetch_results()
    
    # Pick one run to build a baseline template
    benchmarks = geekbenches[0].benchmarks_names()
    workloads = geekbenches[0].workloads_names()
    
    stats  = ["avg", "min", "max"]
    count = len(geekbenches)
    
    print "Parsing {} runs".format(count)

    
    # Initialize stats
    results = {benchmark : 
                        {"min" : sys.maxint, "max" : 0, "avg" : 0} 
               for benchmark in benchmarks}
    
    # Get all the data
    for benchmark in results.iterkeys():
        for bench in geekbenches:
            score = bench.global_scores()[benchmark]
            
            if score > results[benchmark]["max"]:
                results[benchmark]["max"] = score
                
            if score < results[benchmark]["min"]:
                results[benchmark]["min"] = score
            
            results[benchmark]["avg"] += score
        
        results[benchmark]["avg"] /= count
        
    # Convert data to Dataframe
    data = []

    for benchmark in results.iterkeys():
        row = []
        for stat in stats:
            row.append(results[benchmark][stat])
        data.append(tuple(row))
       
    df = pd.DataFrame(data, index=results.iterkeys(), columns=stats)
    
    return df

In [11]:
display(compare_runs())

IndexError: list index out of range